# Training car detection model using YOLOv8

In [2]:
import os.path
import json
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
import shutil
import random

In [3]:
LABEL_MAPPING = {
    'car': 0,
    'rear_bumper_left': 1,
    'rear_bumper_center': 2,
    'rear_bumper_right': 3,
    'rear_right_tire_front': 4,
    'rear_right_tire_back': 5,
    'rear_left_tire_front': 6,
    'rear_left_tire_back': 7,
    'front_right_tire_front': 8,
    'front_right_tire_back': 9,
    'front_left_tire_front': 10,
    'front_left_tire_back': 11,
    'front_bumper_left': 12,
    'front_bumper_center': 13,
    'front_bumper_right': 14,
    'driver_side_mirror': 15,
    'passenger_side_mirror': 16,
}

In [4]:
def view_features(img_path, xtl, ytl, xbr, ybr, points):
    img = cv2.imread(img_path)
    w = xbr - xtl
    h = ybr - ytl

    cv2.rectangle(img, (xtl, ytl), (xtl+xbr, ytl+ybr), [255,0,0], 1)
    cv2.circle(img,(w, h), 1, [255,0,0], -1)

    visible_color = [0,255,0]
    invisible_color = [0,0, 255]
    for p in points:
        x, y, visible = p 
        cv2.circle(img, (x, y), 1, visible_color if visible else invisible_color, -1)

    cv2.imshow('Image', img) 
    cv2.waitKey(0) 
    cv2.destroyAllWindows() 

### Extract features from video annotations

In [9]:
IMAGE_WIDTH = 1920
IMAGE_HEIGHT = 1080

# Opening JSON file
f = open('../../data/cars/annotations.json')
 
# returns JSON object as 
# a dictionary
data = json.load(f)

out_dir = '../../data/cars/labels/train'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

temp = 0
for item in data['items']:
    label_file = open(os.path.join(out_dir, item['id'] + '.txt'), 'w')

    sorted_annotations = sorted(item['annotations'], key=lambda x: x['label_id'])

    xtl = int(sorted_annotations[0]['bbox'][0])
    ytl = int(sorted_annotations[0]['bbox'][1])
    xbr = int(sorted_annotations[0]['bbox'][2])
    ybr = int(sorted_annotations[0]['bbox'][3])
    w = xbr - xtl
    h = ybr - ytl

    label_file.write('0 {} {} {} {} '.format(str((xtl + (w / 2)) / IMAGE_WIDTH), str((ytl + (h / 2)) / IMAGE_HEIGHT),
                                            str(w / IMAGE_WIDTH), str(h / IMAGE_HEIGHT)))
    points = []
    for i in range(1, 17):
        px = int(sorted_annotations[i]['points'][0])
        py = int(sorted_annotations[i]['points'][1])
        visible = 0 if sorted_annotations[i]['attributes']['occluded'] == True else 2
        # Switch append to view a specific feature using LABEL_MAPPING
        points.append([px, py, visible])
        #points.append([int(sorted_annotations[LABEL_MAPPING['front_right_tire_back']]['points'][0]), int(sorted_annotations[LABEL_MAPPING['front_right_tire_back']]['points'][1]), 2])

    for p_, (px, py, visible) in enumerate(points):
        label_file.write('{} {} {}'.format(px / IMAGE_WIDTH, py / IMAGE_HEIGHT, visible))
        if p_ < len(points) - 1:
            label_file.write(' ')
        else:
            label_file.write('\n')

    img_path = '../../data/cars/images/train/%s.png' % item['id']

    if temp == 10:
        view_features(img_path, xtl, ytl, xbr, ybr, points)

    label_file.close()
    if temp >= 10:
        break
    temp += 1

# Closing file
f.close()

### Split validation data

In [40]:
val_label_folder = '../../data/cars/images/val'
val_image_folder = '../../data/cars/labels/val'
shutil.rmtree(val_label_folder, ignore_errors=True)
shutil.rmtree(val_image_folder, ignore_errors=True)

label_files = os.listdir('../../data/cars/labels/train')
image_files = os.listdir('../../data/cars/images/train')

assert(len(label_files) == len(image_files))

os.makedirs(val_label_folder)
os.makedirs(val_image_folder)

random.shuffle(image_files)
split_index = int(0.8 * len(image_files))

val_files = []
for i in range(split_index, len(image_files)):
    val_files.append(os.path.basename(image_files[i]).split('.')[0])

for file in val_files:
    shutil.copy(f'../../data/cars/images/train/{file}.png', f'../../data/cars/images/val')
    shutil.copy(f'../../data/cars/labels/train/{file}.txt', f'../../data/cars/labels/val')

AssertionError: 